<a href="https://colab.research.google.com/github/jquilapan/covid/blob/master/Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.- Paquetes a utilizar**
Se importan los paquetes que se usarán.

In [67]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR

# **2.- Dataset**
Se declara la url de donde proviene el archivo a importar, en este caso, se extrae del github del Ministerio de Ciencias de Chile.

In [68]:
url = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv'

In [69]:
data = pd.read_csv(url)
data = data[['Fecha', 'Casos nuevos totales']]
data = data.reset_index(drop=True)
data['correlativo'] = np.arange(len(data))
data['Fecha'] = pd.to_datetime(data['Fecha'])
data = data.rename(columns={'Casos nuevos totales':'casos_totales'})

# **3.- Modelo a Utilizar**
Se declara el modelo a utilizar, para este ejemplo, usaremos el modelo Support Vector Regressor.

In [70]:
svr_lin = SVR(C=1e3)

Se separan los números correlativos y la cantidad de casos totales, con lo que se entrena el modelo y se genera una predicción para 7 días.

In [71]:
correlativo = data['correlativo'].tolist()
casos = data['casos_totales'].tolist()
correlativo_2 = np.reshape(correlativo, (len(correlativo), 1))
svr_lin.fit(correlativo_2, casos)
for x in range(7):
  predict_value = [max(correlativo) + 1]
  predict_value = np.reshape(predict_value, (len(predict_value), 1))
  pred = svr_lin.predict(predict_value)[0]
  correlativo.append(max(correlativo)+1)
  casos.append(pred)
forecast = pd.DataFrame(list(zip(correlativo, casos)), columns=['correlativo', 'svr'])

# **4.- Construcción del Futuro**
Se construyen los días futuros para crear la combinación de fecha y cantidad de casos totales predichos.

In [74]:
last_month = max(data['Fecha'].unique())
last_month = pd.to_datetime(last_month)
last_correlativo = max(data.correlativo.unique())
meses_predecir = []
numeros_predecir = []
for dia in range(0, 7):
  meses_predecir.append(last_month + pd.tseries.offsets.DateOffset(days=dia+1))
  numeros_predecir.append(last_correlativo + dia+1)
numeros_meses = pd.DataFrame(list(zip(meses_predecir, numeros_predecir)), columns=['Fecha','correlativo'])
forecast_final = pd.merge(numeros_meses, forecast, left_on='correlativo', right_on='correlativo', how='left')